In [2]:
import numpy as np
from itertools import product
from typing import List, Tuple
import sys
sys.path.append("../RM127")  # add path to the compiled module

import PyDecoder_polar as scl  # this is the Cython wrapper

In [3]:
def RM_generator_matrix(r: int, m: int, variation: str) -> Tuple[np.ndarray, List[Tuple[int]]]:
    
    n = 2 ** m
    monomials = []

    for deg in range(r + 1):
        for bits in product([0, 1], repeat=m):
            if sum(bits) == deg:
                monomials.append(bits)

    G = []
    for mono in monomials:
        row = []
        for i in range(n):
            x = [int(b) for b in bin(i)[2:].rjust(m, '0')]
            val = 1
            for a, xi in zip(mono, x):
                if a == 1:
                    val &= xi
            row.append(val)
        G.append(row)
    
    G = np.array(G, dtype=int)
    
    if variation == "None":
        G_final = G
        monomials_final = monomials
    
    if variation == "punctured":
        # Remove the first column (corresponding to evaluation point 0...0)
        G_final = G[:, 1:]
        monomials_final = monomials

    if variation == "shortened":
        # Remove the first column (corresponding to evaluation point 0...0)
        G_final = G[:, 1:]

        # Remove the first row (corresponding to constant polynomial = all-0 monomial)
        G_final = G_final[1:, :]
        monomials_final = monomials[1:]
        
    return G_final, monomials_final

In [ ]:
# Parameters
r = 1
m = 3

decoder = scl.PyDecoder_polar_SCL(r,m)

# Test with shortened RM(1,3)
variation = "None"
G, _ = RM_generator_matrix(r, m, variation)
failure = 0
sample_num = 100

for i in range(sample_num):
    true_msg = np.random.randint(0, 2, G.shape[0])
    true_cw = (true_msg @ G) % 2
    noisy_cw = (true_cw + np.random.binomial(1, 0, len(true_cw))) % 2
    noisy_y = (-1) ** noisy_cw  # Convert in [-1, 1]
    num_flip = decoder.decode(noisy_y.tolist())
    print("True: ", true_cw, "Noisy: ", noisy_cw, "Num_flip: ", num_flip)
    if num_flip != 0:
        failure += 1
failure_rate = failure / sample_num
print("Failure rate: ", failure_rate)


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
failure = 0
sample_num = 10
scale = 0.99

for i in range(sample_num):
    true_msg = np.random.randint(0, 2, G.shape[0])
    true_cw = (true_msg @ G) % 2
    noisy_cw = (true_cw + np.random.binomial(1, 0.1, len(true_cw))) % 2
    noisy_y = 0.99 * noisy_cw + 1/2 * (1 - scale)
    candidates = rm_list_decode(y, r, m, L=4)
    best = candidates[0][0]
    print(best)
    best = (1 - best) // 2
    print("True: ", true_cw, "Noisy: ", noisy_cw, "Decoded: ", best)
    if not np.array_equal(true_cw, best):
        failure += 1
failure_rate = failure / sample_num
print("Failure rate: ", failure_rate)

# Noisy codeword
noisy_cw = np.random.randint(0, 2, size=2**m)
noisy_cw = ((-1) ** noisy_cw).tolist()  # Convert to {-1, +1} representation
decoded_cw = decoder.decode(noisy_cw)

print("Noisy input: ", noisy_cw)
print("Decoded: ", decoded_cw)

# Create decoder instance